In [1]:
import numpy as np
import math as m
import matplotlib.pyplot as plt

Общий класс метода и утилитарные методы:

In [2]:
import base64
import urllib.parse
import io
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from IPython.display import HTML, display
from abc import ABC, abstractmethod


class Method(ABC):

    # кортеж - (1, 2)
    @abstractmethod
    def result(self) -> tuple:
        pass

    @abstractmethod
    def info(self) -> dict:
        pass


# В качестве особого ключа будет 'title', который будет указывать название метода
titleKey = 'title'
minPointKey = '1. Точка минимума'
minimumKey = '2. Минимум'
iterCountKey = '3. Кол-во итераций'
execCountKey = '4. Кол-во вызовов целевой функции'
gradCountKey = '5. Кол-во вычислений градиента'
imageKey = '6. График приближения'


def dictsToTable(dicts: list[dict]):
    keys = set()
    for dict in dicts:
        for key in dict.keys():
            keys.add(key)

    keys.discard('title')
    keys = list(keys)
    keys.sort()
    # формируем таблицу
    table = '<th></th>'
    for dict in dicts:
        table += '<th>' + dict[titleKey] + '</th>'

    table = '<tr>' + table + '</tr>'

    for key in keys:
        table += '<tr>'
        table += '<th>' + str(key) + '</th>'
        for dict in dicts:
            cellData = ''
            if dict.get(key) != None:
                cellData = str(dict[key])

            table += '<th>' + cellData + '</th>'

        table += '</tr>'

    display(HTML('<table style="border:20px black solid">' + table + '</table>'))


def printMethodsInfo(methods: list[Method]):
    dictsToTable([method.info() for method in methods])


def figureToHtml(fig: Figure):
    imgdata = io.BytesIO()
    fig.savefig(imgdata, format='png', dpi=500)
    imgdata.seek(0)
    data = urllib.parse.quote(base64.b64encode(imgdata.read()).decode())
    return '<img src="data:image/png;base64,%s"/>' % data


def pltToHtml():
    fig = plt.gcf()
    data = figureToHtml(fig)
    # fig.clear()
    return data


def fmtFloat(num: float, eps: float) -> str:
    """
    Возвращает сторку с числом num, округленное до точености eps
    Пример: fmtFloat(0.125, 0.01) -> 0.13
    """
    count = -round(m.log10(eps))
    return f"{num:.{count}f}"

def drawPoints(fig: Figure, fun, points: list):
    minx = min(points, key=lambda x: x[0])[0]
    maxx = max(points, key=lambda x: x[0])[0]
    miny = min(points, key=lambda x: x[1])[1]
    maxy = max(points, key=lambda x: x[1])[1]

    deltax = (maxx - minx) / 10
    deltay = (maxy - miny) / 10

    minx -= deltax
    maxx += deltax
    miny -= deltay
    maxy += deltay

    X = np.linspace(minx, maxx, num=200)
    Y = np.linspace(miny, maxy, num=200)
    X, Y = np.meshgrid(X, Y)
    Z = []
    for i in range(200):
        ZZ = []
        for j in range(200):
            ZZ.append(fun([X[i, j], Y[i, j]]))
        Z.append(ZZ)

    ax = fig.subplots()
    ax.contourf(X, Y, Z)
    ax.plot([x[0] for x in points], [x[1]
            for x in points], marker='o', markersize=3, color='red')


Минимизируемые функции:

In [10]:
def f1(x): return 10 * x[0]**2 - 4*x[0]*x[1] + 7 * \
    x[1]**2 - 4*m.sqrt(5) * (5*x[0]-x[1]) - 16


def f1Grad(x): return [-20 * m.sqrt(5) + 20 * x[0] -
                       4*x[1], 4*m.sqrt(5) - 4*x[0] + 14*x[1]]


def rozenbrok(alpha: float):
    # return lambda x: alpha * (x[0]**2 - x[1])**2 + (x[0]-1)**2
    return lambda x: alpha * np.power(np.power(x[0], 2) - x[1], 2) + np.power(x[0]-1, 2)


def rozenbrokGrad(alpha: float):
    # return lambda x: [-2*(1 - x[0]) - 4*x[0]*(-x[0]**2 + x[1])*alpha, 2*(-x[0]**2 + x[1])*alpha]
    return lambda x: [-2*(1 - x[0]) - 4*x[0]*(-np.power(x[0], 2) + x[1])*alpha, 2*(-np.power(x[0], 2) + x[1])*alpha]


alpha1 = 1
alpha2 = 10

f2 = rozenbrok(alpha1)
f2Grad = rozenbrokGrad(alpha1)

f3 = rozenbrok(alpha2)
f3Grad = rozenbrokGrad(alpha2)


Метод золотого сечения для одномерной оптимизации:

In [19]:
def methodGoldenRatio(fun, a: float, b: float, eps: float):
    tau = (m.sqrt(5) + 1) / 2
    ak, bk = a, b
    lk = bk - ak
    xk1 = bk - (bk - ak) / tau
    xk2 = ak + (bk - ak) / tau
    y1, y2 = fun(xk1), fun(xk2)

    iterCount = 0

    while lk >= eps:
        iterCount += 1

        if y1 >= y2:
            ak = xk1
            xk1 = xk2
            xk2 = ak + bk - xk1
            y1 = y2
            y2 = fun(xk2)
        else:
            bk = xk2
            xk2 = xk1
            xk1 = ak + bk - xk2
            y2 = y1
            y1 = fun(xk1)
        lk = bk - ak
    return (ak + bk) / 2, iterCount


Метод градиентного спуска с дробным шагом:

In [25]:
class GradDescMethod(Method):

    def __init__(self, fun, grad, eps: float, startPoint):
        self.points = []
        self.iterCount = 0
        self.execCount = 0
        self.gradCount = 0

        self.fun = fun

        # Параметры дробления шага
        omega = 0.3
        delta = 0.9
        kappa0 = 1.0

        self.eps = eps

        # Сам метод
        # kappa = kappa0

        xk = startPoint
        self.points.append(xk)
        lk = eps + 1
        while lk > eps:
            self.iterCount += 1

            # TODO
            kappa = kappa0

            gradXk = grad(xk)
            self.gradCount += 1

            fxk = fun(xk)
            self.execCount += 1

            newXk = [xk[0] - gradXk[0]*kappa, xk[1] - gradXk[1]*kappa]
            newFxk = fun(newXk)
            self.execCount += 1

            while newFxk > fxk - omega * kappa * np.linalg.norm(gradXk)**2:
                kappa *= delta
                newXk = [xk[0] - gradXk[0]*kappa, xk[1] - gradXk[1]*kappa]
                newFxk = fun(newXk)
                self.execCount += 1

            lk = np.linalg.norm([newXk[0] - xk[0], newXk[1] - xk[1]])
            xk = newXk
            self.points.append(xk)

        self.minPoint = xk
        self.minimum = fun(xk)

    def result(self) -> tuple:
        return self.minPoint, self.minimum

    def info(self) -> dict:
        fig = plt.figure()
        drawPoints(fig, self.fun, self.points)
        figData = figureToHtml(fig)
        plt.close(fig)

        return {
            titleKey: 'Метод градиентного спуска с дробным шагом',
            minPointKey: '(' + fmtFloat(self.minPoint[0], self.eps) + ', ' + fmtFloat(self.minPoint[1], self.eps) + ')',
            minimumKey: fmtFloat(self.minimum, self.eps),
            iterCountKey: self.iterCount,
            execCountKey: self.execCount,
            gradCountKey: self.gradCount,
            imageKey: figData
        }


Метод наискорейшего спуска:

In [21]:
class StepDescMethod(Method):

    def __init__(self, fun, grad, eps: float, startPoint):
        self.points = []
        self.iterCount = 0
        self.execCount = 0
        self.gradCount = 0

        self.eps = eps

        self.fun = fun

        # Сам метод
        xk = startPoint
        self.points.append(xk)
        lk = eps + 1
        while lk > eps:
            self.iterCount += 1

            gradXk = grad(xk)
            self.gradCount += 1

            lam, addExec = methodGoldenRatio(
                lambda l: fun([xk[0] - gradXk[0]*l, xk[1] - gradXk[1]*l]),
                # TODO вместо 1 можно взять другое число
                0, 1, eps
            )
            self.execCount += addExec

            newXk = [xk[0] - gradXk[0]*lam, xk[1] - gradXk[1]*lam]
            lk = np.linalg.norm([newXk[0] - xk[0], newXk[1] - xk[1]])
            xk = newXk
            self.points.append(xk)

        self.minPoint = xk
        self.minimum = fun(xk)

    def result(self) -> tuple:
        return self.minPoint, self.minimum

    def info(self) -> dict:
        fig = plt.figure()
        drawPoints(fig, self.fun, self.points)
        figData = figureToHtml(fig)
        # fig.clf()
        plt.close(fig)

        return {
            titleKey: 'Метод наискорейшего спуска',
            minPointKey: '(' + fmtFloat(self.minPoint[0], self.eps) + ', ' + fmtFloat(self.minPoint[1], self.eps) + ')',
            minimumKey: fmtFloat(self.minimum, self.eps),
            iterCountKey: self.iterCount,
            execCountKey: self.execCount,
            gradCountKey: self.gradCount,
            imageKey: figData
        }


Тесты:

In [30]:
epss = [0.01, 1e-6]

funs = [['Квадратичная функция', f1, f1Grad], ['Функция Розенброка с \u03b1 = 1', f2, f2Grad], ['Функция Розенброка с \u03b1 = 10', f3, f3Grad]]

for fun in funs:
    for eps in epss:
        print(fun[0] + ', \u03b5 = ' + str(eps))

        # TODO менять начальные точки
        res1 = GradDescMethod(fun[1], fun[2], eps, [10, 10])
        res2 = StepDescMethod(fun[1], fun[2], eps, [10, 10])
        printMethodsInfo([res1, res2])

Квадратичная функция, ε = 0.01


Квадратичная функция, ε = 1e-06


Функция Розенброка с α = 1, ε = 0.01


Функция Розенброка с α = 1, ε = 1e-06


Функция Розенброка с α = 10, ε = 0.01


/tmp/ipykernel_1597/2838449083.py:11: RuntimeWarning: overflow encountered in power
  return lambda x: alpha * np.power(np.power(x[0], 2) - x[1], 2) + np.power(x[0]-1, 2)
/tmp/ipykernel_1597/2838449083.py:16: RuntimeWarning: overflow encountered in double_scalars
  return lambda x: [-2*(1 - x[0]) - 4*x[0]*(-np.power(x[0], 2) + x[1])*alpha, 2*(-np.power(x[0], 2) + x[1])*alpha]
/tmp/ipykernel_1597/1996303082.py:24: RuntimeWarning: invalid value encountered in double_scalars
  lambda l: fun([xk[0] - gradXk[0]*l, xk[1] - gradXk[1]*l]),
/tmp/ipykernel_1597/1996303082.py:30: RuntimeWarning: invalid value encountered in double_scalars
  newXk = [xk[0] - gradXk[0]*lam, xk[1] - gradXk[1]*lam]
/tmp/ipykernel_1597/2838449083.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return lambda x: alpha * np.power(np.power(x[0], 2) - x[1], 2) + np.power(x[0]-1, 2)


Функция Розенброка с α = 10, ε = 1e-06
